# Models

### Pre-defined models

Brain-Score includes many user-submitted models that can all be treated like a primate subject.
Thanks to the unified model interface, all models can be used in the same way, and dependencies are automatically installed.

In [1]:
from brainscore_vision import load_model, load_stimulus_set
from brainscore_vision.model_interface import BrainModel

model = load_model('alexnet')
model.start_recording(recording_target=BrainModel.RecordingTarget.V1, time_bins=[(100, 200)])
stimuli = load_stimulus_set('FreemanZiemba2013.aperture-public')  # load some images for the model to look at
neural_predictions = model.look_at(stimuli)
neural_predictions

activations:   0%|          | 0/192 [00:00<?, ?it/s]

layer packaging:   0%|          | 0/1 [00:00<?, ?it/s]

<xarray.NeuroidAssembly (neuroid: 46656, presentation: 135)>
array([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.0477892, 0.0477892, 0.0477892, ..., 0.0477892, 0.0477892,
        0.0477892]], dtype=float32)
Coordinates:
    time_bin                   object (100, 200)
  * neuroid                    (neuroid) MultiIndex
  - neuroid_num                (neuroid) int64 0 1 2 3 ... 46653 46654 46655
  - model                      (neuroid) object 'alexnet' ... 'alexnet'
  - layer                      (neuroid) object 'features.2' ... 'features.2'
  - channel                    (neuroid) int64 0 0 0 0 0 0 ... 63 63 63 63 63 63
  - channel_x                  (neuroid) int64 0 0 0 0 0 0 ... 26 26 26 26 26 26
  - channel_y                  (neuroid) int64 0 1 2 3 4 5 ... 21 22 23 24 25 26
  - neuroid_id                 (neuroid) object 'alexnet.features.2.0' ... 'a...
  - region                     (neuroid) object 'V1' 'V1' 'V1' ... 'V1' 'V1'
  * presentation               (presentation) MultiIndex
  - stimulus_id                (presentation) object '21041db1f26c142812a6627...
  - image_id                   (presentation) object '21041db1f26c142812a6627...
  - sample                     (presentation) int64 5 15 3 13 12 ... 7 8 6 6 10
  - filename                   (presentation) object 'noise-320x320-im38-smp5...
  - image_file_sha1            (presentation) object '21041db1f26c142812a6627...
  - image_file_name            (presentation) object 'noise-320x320-im38-smp5...
  - texture_family             (presentation) int64 38 393 30 18 ... 38 56 48 30
  - resolution                 (presentation) object '320x320' ... '320x320'
  - id                         (presentation) int64 133032 133033 ... 133473
  - image_id_without_aperture  (presentation) object '6ddcf4b55f9151ac7725070...
  - texture_type               (presentation) object 'noise' ... 'texture'

The resulting assembly contains the V1 predictions of the model (46656 neural sites) in response to 135 images. It also contains a lot of metadata, e.g. which layer and channel each neural site comes from, or the type of each image.

## Custom models

You can also define your own models. All models are organized as plugins inside `brainscore_vision/models`.
To interface with Brain-Score, every model has to implement the :ref:`interface`.
Not every method has to be implemented, e.g. a behavior-only model can elect to not implement `start_recording`, and a neuron-only model can elect not to implement `start_task`. But in order to be tested on the full set of benchmarks, all methods have to be implemented.

In [2]:
from brainscore_vision import load_stimulus_set
from typing import List, Tuple

import numpy as np
from numpy.random import RandomState

from brainio.assemblies import DataAssembly, NeuroidAssembly
from brainio.stimuli import StimulusSet
from brainscore_vision.benchmark_helpers.screen import place_on_screen
from brainscore_vision.model_interface import BrainModel


class RandomV1Model(BrainModel):
    def __init__(self):
        self._num_neurons = 50
        # keep track of which time we are outputting
        self._time_bin_start = None
        self._time_bin_end = None

    def look_at(self, stimuli: StimulusSet, **kwargs) -> NeuroidAssembly:
        print(f"Looking at {len(stimuli)} stimuli")
        rnd = RandomState(0)
        recordings = DataAssembly(rnd.rand(len(stimuli), self._num_neurons, 1),
                                  coords={'stimulus_id': ('presentation', stimuli['stimulus_id']),
                                          'texture_type': ('presentation', stimuli['texture_type']),  # in practice we would include all stimulus metadata
                                          'neuroid_id': ('neuroid', np.arange(self._num_neurons)),
                                          'region': ('neuroid', ['V1'] * self._num_neurons),
                                          'time_bin_start': ('time_bin', [self._time_bin_start]),
                                          'time_bin_end': ('time_bin', [self._time_bin_end])},
                                  dims=['presentation', 'neuroid', 'time_bin'])
        recordings.name = 'random_v1_model'
        return recordings

    def start_task(self, task, **kwargs):
        raise NotImplementedError()

    def start_recording(self, recording_target=BrainModel.RecordingTarget, time_bins=List[Tuple[int]]):
        print(f"Recording from {recording_target} during {time_bins} ms")
        if str(recording_target) != BrainModel.RecordingTarget.V1:
            raise NotImplementedError(f"RandomV1Model only supports V1, not {recording_target}")
        if len(time_bins) != 1:
            raise NotImplementedError(f"RandomV1Model only supports a single start-end time-bin, not {time_bins}")
        time_bins = time_bins[0]
        self._time_bin_start, self._time_bin_end = time_bins[0], time_bins[1]

    def visual_degrees(self):
        print("Declaring model to have a visual field size of 8 degrees")
        return 8


model = RandomV1Model()

model.start_recording(BrainModel.RecordingTarget.V1, time_bins=[(100, 200)])
stimuli = load_stimulus_set('FreemanZiemba2013.aperture-public')
stimuli = place_on_screen(stimuli, target_visual_degrees=4, source_visual_degrees=model.visual_degrees())  # adapt visual degree presentation
model.look_at(stimuli)

Recording from V1 during [(100, 200)] ms
Declaring model to have a visual field size of 8 degrees
Looking at 135 stimuli


<xarray.DataAssembly 'random_v1_model' (presentation: 135, neuroid: 50, time_bin: 1)>
array([[[0.5488135 ],
        [0.71518937],
        [0.60276338],
        ...,
        [0.1289263 ],
        [0.31542835],
        [0.36371077]],

       [[0.57019677],
        [0.43860151],
        [0.98837384],
        ...,
        [0.02010755],
        [0.82894003],
        [0.00469548]],

       [[0.67781654],
        [0.27000797],
        [0.73519402],
        ...,
...
        ...,
        [0.82525654],
        [0.00982266],
        [0.87313113]],

       [[0.10649754],
        [0.12570095],
        [0.16622886],
        ...,
        [0.46703886],
        [0.2391275 ],
        [0.44822579]],

       [[0.83378047],
        [0.25050582],
        [0.91409279],
        ...,
        [0.36551811],
        [0.39746763],
        [0.91298598]]])
Coordinates:
  * presentation    (presentation) MultiIndex
  - stimulus_id     (presentation) object '21041db1f26c142812a66277c2957fb3e2...
  - texture_type    (presentation) object 'noise' 'texture' ... 'texture'
  * neuroid         (neuroid) MultiIndex
  - neuroid_id      (neuroid) int64 0 1 2 3 4 5 6 7 ... 42 43 44 45 46 47 48 49
  - region          (neuroid) object 'V1' 'V1' 'V1' 'V1' ... 'V1' 'V1' 'V1' 'V1'
  * time_bin        (time_bin) MultiIndex
  - time_bin_start  (time_bin) int64 100
  - time_bin_end    (time_bin) int64 200